In [1]:
from IPython.core.display import display, HTML
display(HTML("<style>.container { width:90% !important; }</style>"))

In [2]:
from pathlib import Path
import numpy as np
import pandas as pd
import torch
import matplotlib.pyplot as plt

In [3]:
indices = pd.MultiIndex.from_frame(pd.read_csv((Path.cwd().parent / 'data' / 'processed' / 'locations.csv'), usecols=['Time', 'Vehicle_Number_Plate'], parse_dates=['Time'])).reorder_levels([1,0])
area_centers = pd.read_csv((Path.cwd().parent / 'data' / 'processed' / 'areas.csv'), index_col=0)
demand = pd.read_csv((Path.cwd().parent / 'data' / 'processed' / 'demand.csv'), index_col=0)
locations = pd.read_csv((Path.cwd().parent / 'data' / 'processed' / 'locations.csv'), parse_dates=['Time'])
actions = pd.read_csv((Path.cwd().parent / 'data' / 'processed' / 'actions.csv'), parse_dates=['Time'])

In [4]:
demand.index = pd.to_datetime(demand.index, format='%Y-%m-%d %H:%M')
locations.index = pd.MultiIndex.from_frame(locations.loc[:,['Time', 'Vehicle_Number_Plate']])
actionsi = pd.MultiIndex.from_frame(actions.loc[:,['Time', 'Vehicle_Number_Plate']])
actions.index = actionsi
locations.drop(labels=['Time', 'Vehicle_Number_Plate'], axis=1, inplace=True)
actions.drop(labels=['Time', 'Vehicle_Number_Plate'], axis=1, inplace=True)
vehicle_counts = locations.loc[:, locations.columns[locations.columns.str.contains('Zone')]].groupby('Time').sum()

In [5]:
rental = pd.read_csv((Path.cwd().parent / 'data' / 'interim' / 'rental.csv'), index_col=0, parse_dates=['Start_Datetime_Local', 'End_Datetime_Local'])

In [6]:
actionsi[52]

(Timestamp('2020-02-03 12:56:26'), 'BZ23114')

In [7]:
cj=rental.loc[actionsi[52][1]]

In [8]:
cj[(cj['Start_Datetime_Local'].dt.day==6) & (cj['Start_Datetime_Local'].dt.month==2) &
   (cj['Start_Datetime_Local'].dt.year==2020) & (cj['Start_Datetime_Local'].dt.hour>=9)].sort_values(by='Start_Datetime_Local')[:4]

,Vehicle_Engine_Type,Vehicle_Model,Revenue_Net,Start_Datetime_Local,End_Datetime_Local,Start_GPS_Latitude,Start_GPS_Longitude,End_GPS_Latitude,End_GPS_Longitude,Package_Description,Operation_State_Name_Before,Operation_State_Name_After,Reservation_YN,Prebooking_YN,Servicedrive_YN,Start_Zone_Name,End_Zone_Name,Virtual_Start_Zone_Name,Virtual_End_Zone_Name
Vehicle_Number_Plate,,,,,,,,,,,,,,,,,,,
BZ23114,CE,MINI_5-TUERER,0.0,2020-02-06 11:53:00,2020-02-06 12:34:00,55.631501,12.599753,55.630123,12.648977,-,Blue,Green,False,False,1,-,DN Airport End,8,6
BZ23114,CE,MINI_5-TUERER,140.8,2020-02-06 20:22:00,2020-02-06 20:56:00,55.630123,12.648977,55.671776,12.387337,-,Orange,Green,False,False,0,DN Airport Start,-,6,15
BZ23114,CE,MINI_5-TUERER,183.2,2020-02-06 21:13:00,2020-02-06 23:07:00,55.671776,12.387337,55.671770,12.387334,-,Orange,Green,False,False,0,-,-,15,15


In [9]:
l=pd.read_csv((Path.cwd().parent / 'data' / 'processed' / 'locations.csv'), usecols=['Time', 'Vehicle_Number_Plate'], parse_dates=['Time'])[['Time', 'Vehicle_Number_Plate']]
a=pd.read_csv((Path.cwd().parent / 'data' / 'processed' / 'actions.csv'), usecols=['Time', 'Vehicle_Number_Plate'], parse_dates=['Time'])[['Time', 'Vehicle_Number_Plate']]

In [16]:
actions

,,Virtual_Zone_Name_0,Virtual_Zone_Name_1,Virtual_Zone_Name_2,Virtual_Zone_Name_3,Virtual_Zone_Name_4,Virtual_Zone_Name_5,Virtual_Zone_Name_6,Virtual_Zone_Name_7,Virtual_Zone_Name_8,Virtual_Zone_Name_9,Virtual_Zone_Name_10,Virtual_Zone_Name_11,Virtual_Zone_Name_12,Virtual_Zone_Name_13,Virtual_Zone_Name_14,Virtual_Zone_Name_15,Virtual_Zone_Name_16,Virtual_Zone_Name_17,Virtual_Zone_Name_18,Virtual_Zone_Name_19
Time,Vehicle_Number_Plate,,,,,,,,,,,,,,,,,,,,
2020-02-01 00:56:26,CL35192,0,0,0,0,0,0,0,0,0,0,0,0,0,0,1,0,0,0,0,0
2020-02-01 01:56:26,CJ79052,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,1,0
2020-02-01 07:56:26,BY93898,0,0,0,0,0,0,0,0,0,0,1,0,0,0,0,0,0,0,0,0
2020-02-01 08:26:26,CJ75240,0,0,1,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0
2020-02-01 09:56:26,CJ75216,0,0,0,0,0,0,1,0,0,0,0,0,0,0,0,0,0,0,0,0
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
2021-05-03 21:26:26,CT84514,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,1
2021-05-03 21:56:26,CX80057,0,0,1,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0
2021-05-03 22:26:26,CL84858,0,0,0,0,0,0,0,0,1,0,0,0,0,0,0,0,0,0,0,0


In [ ]:
pd.merge(locations, actions, left_index=True, right_index=True, how='inner')

In [18]:
pd.merge(a,l,how='inner', on=['Time', 'Vehicle_Number_Plate'])

,Time,Vehicle_Number_Plate
0,2020-02-01 00:56:26,CL35192
1,2020-02-01 01:56:26,CJ79052
2,2020-02-01 07:56:26,BY93898
3,2020-02-01 08:26:26,CJ75240
4,2020-02-01 09:56:26,CJ75216
...,...,...
13546,2021-05-03 20:56:26,CY43459
13547,2021-05-03 21:26:26,CT84514
13548,2021-05-03 21:56:26,CX80057
13549,2021-05-03 22:56:26,CL16672


In [10]:
print('Indices')
it = indices.to_list()
%timeit it[1000]
%timeit indices[1000]
print('Locations')
vloc=locations.values
lloc=list(vloc)
tloc=tuple(vloc)
%timeit torch.tensor(locations.loc[indices[1000]].values)
%timeit torch.tensor(locations.loc[it[1000][0], it[1000][1]].values)
%timeit torch.tensor(locations.iloc[1000].values)
%timeit torch.tensor(vloc[1000])
%timeit torch.tensor(lloc[1000])
%timeit torch.tensor(tloc[1000])
print('Actions')
at = actionsi.to_list()
%timeit indices[1000] in actionsi
%timeit indices[1000] in at
%timeit it[1000] in at
print('Demand')
dt = demand.to_dict('index')
%timeit torch.tensor(demand.loc[indices[1000][0]].values)
%timeit torch.tensor(tuple(dt[indices[1000][0]].values()))
%timeit torch.tensor(tuple(dt[it[1000][0]].values()))
print('Counts')
vc = vehicle_counts.to_dict('index')
%timeit torch.tensor(vehicle_counts.loc[indices[1000][0]].values)
%timeit torch.tensor(tuple(vc[indices[1000][0]].values()))
%timeit torch.tensor(tuple(vc[it[1000][0]].values()))

Indices


KeyboardInterrupt: 

In [ ]:
t = len(pd.read_csv(Path.cwd().parent / 'data' / 'processed' / 'locations.csv', usecols=[0]))
o = len(pd.read_csv(Path.cwd().parent / 'data' / 'processed' / 'actions.csv', usecols=[0]))
pos_weight = (t-o)/o
pos_weight